# Реализация алгоритмов

### Импорт необходимых библиотек

In [1]:
import sys

sys.path.insert(1, "../")

import os
import pathlib
import collections

import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    roc_auc_score,
)

from src.data.dataset_filenames import extract_type_face
from src.data.utils import is_non_zero_file, get_directory_file_list, resize_img
from src.features import (
    haralick_rdwt_features,
    dispersion_feature,
    mean_feature,
    median_feature,
    maximum_feature,
    minimum_feature,
    fraction_pixels_for_interval_feature,
    scatter_feature,
    hist_feauture,
    hist_gradient_feauture,
    ratio_upper_lower_parts_feature,
    haralick_rdwt_features_with_blockwise,
    get_blocks,
    calc_and_get_haralick,
    lbp_hist_features,
)
from src.features.utils import apply_feature_funcs


%matplotlib inline

### Загрузка изображений лиц

In [2]:
path_to_dataset = pathlib.Path(
    pathlib.Path.home(),
    "Projects",
    "ml",
    "datasets",
    "FACE_ANTISPOOFING",
    "FACE_ANTISPOOFING_FACES",
)
path_to_dataset_rgb_faces = path_to_dataset / "rgb"
path_to_dataset_th_faces = path_to_dataset / "thermal"

#### Функция для загрузки изображений датасета с мета-информацией

In [3]:
def load_dataset_objects(path, img_size=(312, 312), cv_mode=cv2.COLOR_BGR2RGB):
    result = {"filenames": [], "images": [], "subclasses": [], "classes": []}

    for root, dirs, files in os.walk(path):
        for filename in files:
            if not filename.startswith("."):
                result["filenames"].append(filename)

                image = cv2.imread(os.path.join(root, filename))
                image = cv2.cvtColor(image, cv_mode)
                image = resize_img(image, img_size)
                result["images"].append(image)

                result["subclasses"].append(extract_type_face(filename))
                result["classes"].append(int("a" in result["subclasses"][-1]))

    return result

#### Загрузка RGB-изображений лиц

In [4]:
rgb_data = load_dataset_objects(path_to_dataset_rgb_faces)

In [5]:
# plt.imshow(rgb_data["images"][0])

#### Загрузка Thermal-изображений лиц

In [6]:
th_data = load_dataset_objects(path_to_dataset_th_faces, cv_mode=cv2.COLOR_BGR2GRAY)

In [7]:
# plt.imshow(th_data["images"][0], cmap="gray")

### Функция для расчета метрик

In [8]:
def FAR_score(y_true, y_pred):
    y_true = y_true.astype(bool)
    y_pred = y_pred.astype(bool)
    return sum(y_pred & ~y_true) / len(y_true[np.where(y_true == False)])

In [9]:
def FRR_score(y_true, y_pred):
    y_true = y_true.astype(bool)
    y_pred = y_pred.astype(bool)
    return sum(~y_pred & y_true) / len(y_true[np.where(y_true == True)])

In [10]:
def HTER_score(y_true, y_pred):
    far = FAR_score(y_true, y_pred)
    frr = FRR_score(y_true, y_pred)
    hter = (far + frr) / 2
    return hter

In [11]:
def get_metric_values(y_true, y_pred, pos_label=1):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(
            y_true, y_pred, average="binary", pos_label=pos_label
        ),
        "recall": recall_score(y_true, y_pred, average="binary", pos_label=pos_label),
        "f1": f1_score(y_true, y_pred, average="binary", pos_label=pos_label),
        "roc_auc": roc_auc_score(
            y_true,
            y_pred,
            average="weighted",
        ),
        "hter": HTER_score(y_true, y_pred),
    }

In [12]:
path_to_metrics_result = "../reports/tables/metrics_report.csv"

In [13]:
if os.path.exists(path_to_metrics_result):
    measured_metrics = pd.read_csv(path_to_metrics_result)
else:
    measured_metrics = pd.DataFrame(
        {"Metric": ["Accuracy", "Precision", "Recall", "F1", "ROC-AUC", "HTER"]}
    )

In [14]:
measured_metrics

,Metric
0,Accuracy
1,Precision
2,Recall
3,F1
4,ROC-AUC
5,HTER


### Алгоритм RDWT-Haralick-SVM (for thermal) without block-wise

In [15]:
X = []
y = th_data["classes"].copy()

for img in th_data["images"]:
    X.append(haralick_rdwt_features(img))

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

measured_metrics[
    "RDWT-Haralick-SVM (for thermal) without block-wise"
] = get_metric_values(y_test, y_pred).values()

In [16]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal) without block-wise
0,Accuracy,0.973856
1,Precision,0.927273
2,Recall,1.000000
3,F1,0.962264
4,ROC-AUC,0.980392
5,HTER,0.019608


### Алгоритм RDWT-Haralick-SVM (for thermal) with block-wise

In [17]:
X = []
y = th_data["classes"].copy()

for img in th_data["images"]:
    X.append(haralick_rdwt_features_with_blockwise(img))

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

measured_metrics["RDWT-Haralick-SVM (for thermal) with block-wise"] = get_metric_values(
    y_test, y_pred
).values()

In [18]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal) without block-wise,RDWT-Haralick-SVM (for thermal) with block-wise
0,Accuracy,0.973856,1.0
1,Precision,0.927273,1.0
2,Recall,1.000000,1.0
3,F1,0.962264,1.0
4,ROC-AUC,0.980392,1.0
5,HTER,0.019608,0.0


### Алгоритм Haralick-SVM (for thermal)

In [19]:
X = []
y = th_data["classes"].copy()

for img in th_data["images"]:
    X.append(calc_and_get_haralick(img))

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

measured_metrics["Haralick-SVM (for thermal)"] = get_metric_values(
    y_test, y_pred
).values()

In [20]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal) without block-wise,RDWT-Haralick-SVM (for thermal) with block-wise,Haralick-SVM (for thermal)
0,Accuracy,0.973856,1.0,0.973856
1,Precision,0.927273,1.0,0.960784
2,Recall,1.000000,1.0,0.960784
3,F1,0.962264,1.0,0.960784
4,ROC-AUC,0.980392,1.0,0.970588
5,HTER,0.019608,0.0,0.029412


### Алгоритм RDWT-Haralick-SVM (for GrayScale of RGB) without block-wise

In [21]:
X = []
y = rgb_data["classes"].copy()

for img in rgb_data["images"]:
    X.append(haralick_rdwt_features(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)))

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

measured_metrics[
    "RDWT-Haralick-SVM (for GrayScale of RGB) without block-wise"
] = get_metric_values(y_test, y_pred).values()

In [22]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal) without block-wise,RDWT-Haralick-SVM (for thermal) with block-wise,Haralick-SVM (for thermal),RDWT-Haralick-SVM (for GrayScale of RGB) without block-wise
0,Accuracy,0.973856,1.0,0.973856,0.993464
1,Precision,0.927273,1.0,0.960784,0.981132
2,Recall,1.000000,1.0,0.960784,1.000000
3,F1,0.962264,1.0,0.960784,0.990476
4,ROC-AUC,0.980392,1.0,0.970588,0.995050
5,HTER,0.019608,0.0,0.029412,0.004950


### Алгоритм RDWT-Haralick-SVM (for GrayScale of RGB) with block-wise

In [23]:
X = []
y = rgb_data["classes"].copy()

for img in rgb_data["images"]:
    X.append(
        haralick_rdwt_features_with_blockwise(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY))
    )

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

measured_metrics[
    "RDWT-Haralick-SVM (for GrayScale of RGB) with block-wise"
] = get_metric_values(y_test, y_pred).values()

In [24]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal) without block-wise,RDWT-Haralick-SVM (for thermal) with block-wise,Haralick-SVM (for thermal),RDWT-Haralick-SVM (for GrayScale of RGB) without block-wise,RDWT-Haralick-SVM (for GrayScale of RGB) with block-wise
0,Accuracy,0.973856,1.0,0.973856,0.993464,1.0
1,Precision,0.927273,1.0,0.960784,0.981132,1.0
2,Recall,1.000000,1.0,0.960784,1.000000,1.0
3,F1,0.962264,1.0,0.960784,0.990476,1.0
4,ROC-AUC,0.980392,1.0,0.970588,0.995050,1.0
5,HTER,0.019608,0.0,0.029412,0.004950,0.0


### Алгоритм Haralick-SVM (for GrayScale of RGB)

In [25]:
X = []
y = rgb_data["classes"].copy()

for img in rgb_data["images"]:
    X.append(calc_and_get_haralick(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)))

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

measured_metrics["Haralick-SVM (for GrayScale of RGB)"] = get_metric_values(
    y_test, y_pred
).values()

In [26]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal) without block-wise,RDWT-Haralick-SVM (for thermal) with block-wise,Haralick-SVM (for thermal),RDWT-Haralick-SVM (for GrayScale of RGB) without block-wise,RDWT-Haralick-SVM (for GrayScale of RGB) with block-wise,Haralick-SVM (for GrayScale of RGB)
0,Accuracy,0.973856,1.0,0.973856,0.993464,1.0,0.856209
1,Precision,0.927273,1.0,0.960784,0.981132,1.0,0.788462
2,Recall,1.000000,1.0,0.960784,1.000000,1.0,0.788462
3,F1,0.962264,1.0,0.960784,0.990476,1.0,0.788462
4,ROC-AUC,0.980392,1.0,0.970588,0.995050,1.0,0.839775
5,HTER,0.019608,0.0,0.029412,0.004950,0.0,0.160225


### Алгоритм HeuristicFeatures-SVM (for Thermal)

In [27]:
X = []
y = th_data["classes"].copy()

feature_funcs = [
    dispersion_feature,
    mean_feature,
    median_feature,
    maximum_feature,
    #                 minimum_feature,
    #                 fraction_pixels_for_interval_feature,
    scatter_feature,
    #                 hist_feauture,
    #             hist_gradient_feauture,
    #             ratio_upper_lower_parts_feature,
]

for img in th_data["images"]:
    X.append(apply_feature_funcs(img, feature_funcs))

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

measured_metrics["HeuristicFeatures-SVM (for Thermal)"] = get_metric_values(
    y_test, y_pred
).values()

In [28]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal) without block-wise,RDWT-Haralick-SVM (for thermal) with block-wise,Haralick-SVM (for thermal),RDWT-Haralick-SVM (for GrayScale of RGB) without block-wise,RDWT-Haralick-SVM (for GrayScale of RGB) with block-wise,Haralick-SVM (for GrayScale of RGB),HeuristicFeatures-SVM (for Thermal)
0,Accuracy,0.973856,1.0,0.973856,0.993464,1.0,0.856209,0.980392
1,Precision,0.927273,1.0,0.960784,0.981132,1.0,0.788462,0.961538
2,Recall,1.000000,1.0,0.960784,1.000000,1.0,0.788462,0.980392
3,F1,0.962264,1.0,0.960784,0.990476,1.0,0.788462,0.970874
4,ROC-AUC,0.980392,1.0,0.970588,0.995050,1.0,0.839775,0.980392
5,HTER,0.019608,0.0,0.029412,0.004950,0.0,0.160225,0.019608


### Алгоритм HeuristicFeatures-SVM (for GrayScale of RGB)

In [29]:
X = []
y = rgb_data["classes"].copy()

feature_funcs = [
    dispersion_feature,
    mean_feature,
    median_feature,
    maximum_feature,
    #                 minimum_feature,
    #                 fraction_pixels_for_interval_feature,
    scatter_feature,
    #                 hist_feauture,
    #             hist_gradient_feauture,
    #             ratio_upper_lower_parts_feature,
]

for img in rgb_data["images"]:
    X.append(apply_feature_funcs(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), feature_funcs))


X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

measured_metrics["HeuristicFeatures-SVM (for GrayScale of RGB)"] = get_metric_values(
    y_test, y_pred
).values()

In [30]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal) without block-wise,RDWT-Haralick-SVM (for thermal) with block-wise,Haralick-SVM (for thermal),RDWT-Haralick-SVM (for GrayScale of RGB) without block-wise,RDWT-Haralick-SVM (for GrayScale of RGB) with block-wise,Haralick-SVM (for GrayScale of RGB),HeuristicFeatures-SVM (for Thermal),HeuristicFeatures-SVM (for GrayScale of RGB)
0,Accuracy,0.973856,1.0,0.973856,0.993464,1.0,0.856209,0.980392,0.647059
1,Precision,0.927273,1.0,0.960784,0.981132,1.0,0.788462,0.961538,0.485714
2,Recall,1.000000,1.0,0.960784,1.000000,1.0,0.788462,0.980392,0.653846
3,F1,0.962264,1.0,0.960784,0.990476,1.0,0.788462,0.970874,0.557377
4,ROC-AUC,0.980392,1.0,0.970588,0.995050,1.0,0.839775,0.980392,0.648705
5,HTER,0.019608,0.0,0.029412,0.004950,0.0,0.160225,0.019608,0.351295


### Алгоритм LBP-SVM (for Thermal)

In [31]:
X = []
y = th_data["classes"].copy()

for img in th_data["images"]:
    X.append(lbp_hist_features(img, 40, 80))

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

measured_metrics["LBP-SVM (for Thermal)"] = get_metric_values(y_test, y_pred).values()

In [32]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal) without block-wise,RDWT-Haralick-SVM (for thermal) with block-wise,Haralick-SVM (for thermal),RDWT-Haralick-SVM (for GrayScale of RGB) without block-wise,RDWT-Haralick-SVM (for GrayScale of RGB) with block-wise,Haralick-SVM (for GrayScale of RGB),HeuristicFeatures-SVM (for Thermal),HeuristicFeatures-SVM (for GrayScale of RGB),LBP-SVM (for Thermal)
0,Accuracy,0.973856,1.0,0.973856,0.993464,1.0,0.856209,0.980392,0.647059,1.0
1,Precision,0.927273,1.0,0.960784,0.981132,1.0,0.788462,0.961538,0.485714,1.0
2,Recall,1.000000,1.0,0.960784,1.000000,1.0,0.788462,0.980392,0.653846,1.0
3,F1,0.962264,1.0,0.960784,0.990476,1.0,0.788462,0.970874,0.557377,1.0
4,ROC-AUC,0.980392,1.0,0.970588,0.995050,1.0,0.839775,0.980392,0.648705,1.0
5,HTER,0.019608,0.0,0.029412,0.004950,0.0,0.160225,0.019608,0.351295,0.0


### Алгоритм LBP-SVM (for GrayScale of RGB)

In [33]:
X = []
y = rgb_data["classes"].copy()

for img in rgb_data["images"]:
    X.append(lbp_hist_features(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), 40, 80))

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

measured_metrics["LBP-SVM (for GrayScale of RGB)"] = get_metric_values(
    y_test, y_pred
).values()

In [34]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal) without block-wise,RDWT-Haralick-SVM (for thermal) with block-wise,Haralick-SVM (for thermal),RDWT-Haralick-SVM (for GrayScale of RGB) without block-wise,RDWT-Haralick-SVM (for GrayScale of RGB) with block-wise,Haralick-SVM (for GrayScale of RGB),HeuristicFeatures-SVM (for Thermal),HeuristicFeatures-SVM (for GrayScale of RGB),LBP-SVM (for Thermal),LBP-SVM (for GrayScale of RGB)
0,Accuracy,0.973856,1.0,0.973856,0.993464,1.0,0.856209,0.980392,0.647059,1.0,0.993464
1,Precision,0.927273,1.0,0.960784,0.981132,1.0,0.788462,0.961538,0.485714,1.0,1.000000
2,Recall,1.000000,1.0,0.960784,1.000000,1.0,0.788462,0.980392,0.653846,1.0,0.980769
3,F1,0.962264,1.0,0.960784,0.990476,1.0,0.788462,0.970874,0.557377,1.0,0.990291
4,ROC-AUC,0.980392,1.0,0.970588,0.995050,1.0,0.839775,0.980392,0.648705,1.0,0.990385
5,HTER,0.019608,0.0,0.029412,0.004950,0.0,0.160225,0.019608,0.351295,0.0,0.009615


In [35]:
measured_metrics.to_csv(path_to_metrics_result, index=False)  